<a href="https://colab.research.google.com/github/CodyBurker/W266_Group_Project/blob/CodyCNNBase/CodyBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setup for Google Colab
!pip install transformers
!git clone https://github.com/CodyBurker/W266_Group_Project
!pip install datasets
def read_in_data(path="W266_Group_Project/"):
    import pandas as pd
    X_train = pd.read_csv(path + "x_train_sampled_yelp_data.csv")
    y_train = pd.read_csv(path + "y_train_sampled_yelp_data.csv")
    X_test = pd.read_csv(path + "x_test_sampled_yelp_data.csv")
    y_test = pd.read_csv(path + "y_test_sampled_yelp_data.csv")
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = read_in_data()

     |████████████████████████████████| 3.8 MB 4.1 MB/s 
     |████████████████████████████████| 596 kB 52.6 MB/s 
     |████████████████████████████████| 67 kB 4.0 MB/s 
     |████████████████████████████████| 6.5 MB 32.4 MB/s 
     |████████████████████████████████| 895 kB 52.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'W266_Group_Project'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 50 (delta 17), reused 22 (delta 2), pack-reused 0
Unpacking objects: 100% (50/50), done.
Checking out files: 100% (9/9), done.
     |████████████████████████████████| 311 kB 4.3 MB/s 
     |████████████████████████████████| 134 kB 58.5 MB/s 
     |████████████████████████████████| 1.1 MB 55.8 MB/s 
     |████████████████████████████████| 212 kB 71.5 MB/s 
     |███████████████


Bert Model:[bert-base-multilingual-uncased-sentiment](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment?text=not+a+good+representation+of+modern+filipino+food+the+menu+and+execution+needs+a+lot+of+work+the+korean+in+lotte+is+done+well+and+worth+the+expense+hopefully+taglish+place+will+make+some+adjustments)


Using this: https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209
Also this: https://huggingface.co/docs/datasets/use_dataset

In [2]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
# X_train['encoded'] = X_train['text'][0:100].map(lambda sent: tokenizer(sent,add_special_tokens = True,    truncation = True, padding = "max_length", return_attention_mask = True, return_tensors = "pt") )

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

In [3]:
# Read in dataset from hugging face
from datasets import load_dataset
dataset = load_dataset("codyburker/yelp_review_sampled")

Using custom data configuration codyburker--yelp_review_sampled-dcb174a34fabaa1e


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/50.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/codyburker--yelp_review_sampled-dcb174a34fabaa1e/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
# Following this tutorial
# https://towardsdatascience.com/a-beginners-guide-to-use-bert-for-the-first-time-2e99b8c5423
from transformers import pipeline
# classifier = pipeline('sentiment-analysis')
# result = classifier('We are very happy to show you the 🤗 Transformers library.')
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', tokenizer="nlptown/bert-base-multilingual-uncased-sentiment", device=0)

In [48]:
# Test without any fine-tuning
# Passing arguments from here to avoid an error that keeps popping up: https://github.com/huggingface/transformers/issues/12039
n_records = 20000
results = classifier(dataset['test']['text'][0:n_records], padding=True, truncation=True)

In [49]:
labels = [prediction['label'][0] for prediction in results]

In [50]:
import pandas as pd
predictions = pd.DataFrame({'text':dataset['test']['text'][0:n_records],
              'true':dataset['test']['stars'][0:n_records],
              'predicted':labels
              })
predictions['sq'] = (predictions['true'] - predictions['predicted'].astype(int)) **2
predictions.mean()['sq']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  import sys


0.6701

In [70]:
worst = predictions.sort_values('sq',ascending=False).head(10)

for row in worst.iterrows():
  print("Text:",row[1]['text'])
  print("True rating:", row[1]['true'])
  print("Predicted Rating:", row[1]['predicted'])
  print("")

Text: Autozone installed a new battery for me and afterwards my radio and GPS did not work.  Autozone was mystified and sent me back to the Saab dealer, who wanted $128 to diagnose the problem.  A friend recommended McCoullough as an alternative.  I was in and out in 25 minutes... with a new fuse.  McCullough did not charge me for this service, just asked that I come back.  I will!
True rating: 5
Predicted Rating: 1

Text: Where else can you find a place that will BAKE THE COOKIES FREAH FOR YOU, give you a box of hot cookies and send you on your merry way? NOWHERE THATS WHERE. If I go to Boulder I have to stop at this place, there's no option not to. There is no place like this in Denver and if there is somebody please let me know because I can't find it.
True rating: 5
Predicted Rating: 1

Text: Whether buying lux eye wear, designer footwear or a great pair of uggs or hunter, Voted  Best  By Improper Bostonian 2011, you can find almost anything you crave in fashion. Jackets ,hats, sca